In [1]:
import os
from tqdm import tqdm
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, as_completed
import pickle as pkl
import math
from transformers import AutoModelForCausalLM, GemmaConfig, AutoTokenizer, AutoModel, MistralConfig, MistralModel, MistralForCausalLM, LlamaConfig, LlamaForCausalLM
import torch
import torch.nn as nn
import torch.nn.init as init
import json
import pickle
import pandas as pd

In [2]:
# 10 sentence prediction

model_name_or_path = "/kaggle/input/ps-v2-output/hi_model/checkpoint-52710"  # Update with your model path
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)


In [3]:
def calculate_perplexity_and_generate(model, tokenizer, sentences, max_length=50):
    model.eval()
    perplexities = []
    generated_texts = []

    with torch.no_grad():
        for sentence in sentences:
            # Tokenize the sentence
            inputs = tokenizer(sentence, return_tensors="pt")
            input_ids = inputs.input_ids

            # Forward pass to calculate perplexity
            outputs = model(input_ids, labels=input_ids)
            loss = outputs.loss
            perplexity = math.exp(loss.item())
            perplexities.append(perplexity)

            # Generate text based on the input
            generated_ids = model.generate(
                input_ids, max_length=max_length, do_sample=True
            )
            decoded_text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
            generated_texts.append(decoded_text)

    return perplexities, generated_texts


In [4]:
hindi_sentences = [
    "यह एक उदाहरण वाक्य है।",
    "मुझे किताबें पढ़ना पसंद है।",
    "भारत एक सुंदर देश है।",
    "आज मौसम बहुत अच्छा है।",
    "वह स्कूल जा रहा है।",
    "मैं संगीत सुन रहा हूँ।",
    "यह बहुत कठिन प्रश्न है।",
    "हमें अपना काम समय पर करना चाहिए।",
    "कृपया अपनी सहायता प्रदान करें।",
    "हम कल यात्रा पर जाएंगे।"
]

perplexities, generated_texts = calculate_perplexity_and_generate(model, tokenizer, hindi_sentences)
for i, sentence in enumerate(hindi_sentences):
    print(f"Sentence: {sentence}")
    print(f"Perplexity: {perplexities[i]}")
    print(f"Generated Text: {generated_texts[i]}\n")


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's 

Sentence: यह एक उदाहरण वाक्य है।
Perplexity: 57860.420222356624
Generated Text: यह एक उदाहरण वाक्य है कि हम अमर्त्य सेन: विवादों में यह कार्य जैसे बैंक के भावनात्मक इसके लिए लठरीय सूचनािंग और किसी भी प्रकाशित की जा रही है
eos यह वह चसेनजित व कोसिओगी यह आधुनिक काव्य

Sentence: मुझे किताबें पढ़ना पसंद है।
Perplexity: 63896459.160597876
Generated Text: मुझे किताबें पढ़ना पसंद है ... एक सपना अपने लिए और सबके शनिवार
eos> इसे "समझले ही इस तरह के भी आर्टिकल दश नियंत्रण के लिए यहाँ क्लिक कर काफी पहले लगता है कि किस मिट्टी में यह गर्वी फूलों के पीछे

Sentence: भारत एक सुंदर देश है।
Perplexity: 162681.85668559006
Generated Text: भारत एक सुंदर देश है इस शहर में जहाँ फिर भी यही बात उसके लिये आग है ही...  दिसंबर  में हम इंसान  दिसंबर दिसंबर 2019 शुक्रवार
eos> 2015 मुझे क्यों मारना शायद मालूम नहीं

Sentence: आज मौसम बहुत अच्छा है।
Perplexity: 23641.880885594837
Generated Text: आज मौसम बहुत अच्छा है उस पर चलना चाहता मित्र मे इस बात पर मुझे याद पड़ता पर अपना साथ और फिर जब हमारा सामान देखने पर सोचने–जा